## Imports

In [ ]:
from pathlib import Path
import numpy as np

from flood_adapt import FloodAdapt
from flood_adapt.misc.exceptions import AlreadyExistsError
from flood_adapt.objects import (
    EventSet,
    Projection, 
    PhysicalProjection, 
    SocioEconomicChange, 
    Scenario, 
    Strategy,
    FloodProof,
    SelectionType,
    )
from flood_adapt.objects.forcing.unit_system import UnitTypesLength, UnitfulLength, UnitfulLengthRefValue, VerticalReference
from flood_adapt.config.config import Settings


## Step 1a: Inputs (Event set and SLR)

Provide the event set and the range of SLR values to be calculated. 

In [ ]:
DATA_DIR = Path(r"c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database")
site = "charleston_beta_release"
name_event_set = "event_set_coastal"

slr = np.arange(0, 3.1, 1)
unit = UnitTypesLength.feet
fp_height = 2  # floodproof height 


## Step 1b: Reading FloodAdapt database and event set

In [ ]:
# Setup FloodAdapt
fn_event_set = DATA_DIR / site / "input" / "events" / name_event_set / f"{name_event_set}.toml"
settings = Settings(
    DATABASE_ROOT=DATA_DIR,
    DATABASE_NAME=site
)
fa = FloodAdapt(database_path=settings.database_path)

event_set = EventSet.load_file(fn_event_set)

## Step 2a: Create projections

In [ ]:
# create projections for all SLR values
projections = []
se_change = SocioEconomicChange(population_growth_existing=0,
                          economic_growth=0)
for nn,s in enumerate(slr):
    phys_proj = PhysicalProjection(sea_level_rise=UnitfulLength(value=s, units=unit))
    proj = Projection(
        name=f"SLR_{nn}",
        description=f"Sea level rise of {s:.2f} in {unit}",
        physical_projection=phys_proj, 
        socio_economic_change=se_change
    )
    
    try:
        fa.save_projection(proj)
    except AlreadyExistsError:
        print(f"Projection {proj.name} already exists in database")
    projections.append(proj)

projections

## Step 2b: Create strategy

In [ ]:
# Create a strategy object

flood_proof = FloodProof(
    name="floodproof_all",
    description="Floodproofing all buildings.",
    selection_type=SelectionType.all,
    property_type="ALL",
    elevation=UnitfulLength(value=fp_height, units=unit)
)

strategies = [
    Strategy(
        name="no_measures",
        description="No measures",
        measures=[],
    ),
    Strategy(
        name="floodproof_all",
        description="Floodproof all buildings",
        measures=[flood_proof],
    )
]

for strat in strategies:
    try: 
        fa.save_strategy(strat)
    except AlreadyExistsError:
        print("Strategy already exists in database")

strategies

## Step 3: Set up scenarios 

In [ ]:
scenarios = []
for strat in strategies:
    for proj in projections:
        for event in event_set.sub_events:
            scen = Scenario(
                name=f"{proj.name}_{event.name}_{strat.name}",
                description=f"Scenario with {proj.description}, Event {event.name}, and {strat.description}",
                event=event.name,
                projection=proj.name,
                strategy=strat.name,
            )
            try: 
                fa.save_scenario(scen)
            except AlreadyExistsError:
                print("Scenario already exists in database")
            scenarios.append(scen)

## Step 4: Run scenarios